## implementing LDA with gensim
1) import dataset
2) preprocess text
3) create gensim dictionary and corpus
4) build the topic model
5) analyze

In [1]:
# Only run this once
import nltk
nltk.download('all')

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to
[nltk_data]    |     C:\Users\Whyve\AppData\Roaming\nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to
[nltk_data]    |     C:\Users\Whyve\AppData\Roaming\nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     C:\Users\Whyve\AppData\Roaming\nltk_data...
[nltk_data]    |   Package biocreative_ppi is already up-to-date!
[nltk_data]    | Downloading package brown to
[nltk_data]    |     C:\Users\Whyve\AppData\Roaming\nltk_data...
[nltk_data]    |   Package brown is already up-to-date!
[nltk_data]    | Downloading package brown_tei to
[nltk_data]    |     C:\Users\Whyve\AppData\Roaming\nltk_data...
[nltk_data]    |   Package brown_tei is already up-to-date!
[nltk_data]    | Downloading package cess_cat to
[nltk_data]    |     C:\Users\W

[nltk_data]    |   Package product_reviews_1 is already up-to-date!
[nltk_data]    | Downloading package product_reviews_2 to
[nltk_data]    |     C:\Users\Whyve\AppData\Roaming\nltk_data...
[nltk_data]    |   Package product_reviews_2 is already up-to-date!
[nltk_data]    | Downloading package pros_cons to
[nltk_data]    |     C:\Users\Whyve\AppData\Roaming\nltk_data...
[nltk_data]    |   Package pros_cons is already up-to-date!
[nltk_data]    | Downloading package qc to
[nltk_data]    |     C:\Users\Whyve\AppData\Roaming\nltk_data...
[nltk_data]    |   Package qc is already up-to-date!
[nltk_data]    | Downloading package reuters to
[nltk_data]    |     C:\Users\Whyve\AppData\Roaming\nltk_data...
[nltk_data]    |   Package reuters is already up-to-date!
[nltk_data]    | Downloading package rte to
[nltk_data]    |     C:\Users\Whyve\AppData\Roaming\nltk_data...
[nltk_data]    |   Package rte is already up-to-date!
[nltk_data]    | Downloading package semcor to
[nltk_data]    |     C:\

[nltk_data]    |   Package word2vec_sample is already up-to-date!
[nltk_data]    | Downloading package panlex_swadesh to
[nltk_data]    |     C:\Users\Whyve\AppData\Roaming\nltk_data...
[nltk_data]    |   Package panlex_swadesh is already up-to-date!
[nltk_data]    | Downloading package mte_teip5 to
[nltk_data]    |     C:\Users\Whyve\AppData\Roaming\nltk_data...
[nltk_data]    |   Package mte_teip5 is already up-to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     C:\Users\Whyve\AppData\Roaming\nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger is already up-
[nltk_data]    |       to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     C:\Users\Whyve\AppData\Roaming\nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger_ru is already
[nltk_data]    |       up-to-date!
[nltk_data]    | Downloading package perluniprops to
[nltk_data]    |     C:\Users\Whyve\AppData\Roamin

True

In [2]:
import numpy as np
import pandas as pd
import re
import os
import string
import logging
from pprint import pprint
from collections import defaultdict # for pos tag -> wordnet tag

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.tag import pos_tag
from nltk.corpus import wordnet
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer

import gensim
import pyLDAvis.gensim
from gensim.models import Phrases # For adding n-grams
from gensim import corpora, models, similarities 

logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.DEBUG)

In [5]:
# Perform topic modelling with the help of nltk and gensim 
#
class TopicModeler:
    # Initialize an instance of the TopicModeler class
    # params:
    #   corpus<numpy.ndarray([string])>: A numpy array of strings, representing documents in a corpus
    #
    def __init__(self, corpus):
        self.corpus = np.copy(corpus)
        self.dictionary = None
        self.model = None
        self.N = np.shape(self.corpus)[0]
        logging.info('Initialized a TopicModeler with corpus size N=' + str(self.N))
        return
        
        
    # Split the documents in the corpus into lists of raw tokens
    #
    def tokenize(self):
        logging.info('Performing tokenization.')

        # Convert each document in the corpus into a list of tokens 
        for document in range(self.N):
            self.corpus[document] = word_tokenize(self.corpus[document])
        return
    
    
    # Perform morphological normalization on the corpus as a list of raw tokens
    # params:
    #   norm<'lemma'|'stem'>: The form of morphological normalization to use
    #
    def normalize(self, norm='lemma'):
        logging.info('Performing normalization.')
        logging.debug('norm=' + str(norm))
        
        if (norm == 'lemma'):
            logging.info('Performing lemmatization.')
            
            lemmatizer = WordNetLemmatizer()
            
            # Maps nltk pos tags into wordnet pos tags
            tag_map = defaultdict(lambda : wordnet.NOUN)
            tag_map['J'] = wordnet.ADJ
            tag_map['V'] = wordnet.VERB
            tag_map['R'] = wordnet.ADV
            
            self.corpus = [[lemmatizer.lemmatize(tagged_token[0], tag_map[tagged_token[1][0]])
                            for tagged_token in pos_tag(document)]
                               for document in self.corpus]
        elif (norm == 'stem'):
            logging.info('Performing stemming.')
            stemmer = PorterStemmer()
            self.corpus = [[stemmer.stem(token) for token in document] for document in self.corpus]
        else:
            logging.warning('Invalid parameter! Skipping normalization...')
        return
            
        
    # Filter out any tokens that are not within a specified string length
    # params:
    #   min_strlen<integer>: The minimum amount of chars a token can have
    #   max_strlen<integer>: The maximum amount of chars a token can have
    #
    def filter_length(self, min_strlen=1, max_strlen=100):
        logging.info('Filtering out tokens that are out of the length bounds.')
        logging.debug('min_strlen=' + str(min_strlen))
        logging.debug('max_strlen=' + str(max_strlen))
        
        self.corpus = [[token for token in document
                            if len(token) >= min_strlen
                            and len(token) <= max_strlen
                       ] for document in self.corpus]
        return
    
    
    # Filter out any tokens that match a regular expression
    # params:
    #   pattern<raw string>: The regular expresstion to match
    #
    def filter_match(self, pattern):
        logging.info('Filtering out tokens that match the regular expression: ' + str(pattern))
        
        self.corpus = [[token for token in document
                            if re.search(pattern, token)
                       ] for document in self.corpus]
    
    
    # Lowercase all tokens
    #
    def lowercase(self):
        logging.info('Converting all tokens to lowercase.')
        self.corpus = [[token.lower() for token in document] for document in self.corpus]
        
        
    # Add n-grams to the corpus
    # params:
    #   n<integer(2|3)>: The maximum length of the sequence of words to add to the corpus
    #   min_count<integer>: The minimum amount of token occurances needed for an n-gram to be included
    #
    def add_n_grams(self, n=2, min_count=1):
        logging.info('Performing normalization.')
        logging.debug('n=' + str(n))
        logging.debug('min_count=' + str(min_count))

        logging.info('Adding 2-grams')
        bigram = Phrases(self.corpus, min_count=min_count, delimiter=b' ')
        
        if n == 3:
            trigram = Phrases(bigram[self.corpus], min_count=1, delimiter=b' ')
            for document in range(self.N):
                self.corpus[document] = [n_gram for n_gram in trigram[bigram[self.corpus[document]]] 
                               if n_gram.count(' ') < n]
        elif n == 2:
            for document in range(self.N):
                self.corpus[document] = [n_gram for n_gram in bigram[self.corpus[document]]
                               if n_gram.count(' ') < n]
        else:
            logging.warning('Invalid parameter! Skipping n-grams...')
        return
    
    
    # Remove stop-words
    # params:
    #   stop<list([string])>: A list containing all stop words to exclude
    #
    def remove_stop_words(self, stop=stopwords.words('english')):
        logging.info('Removing stop-words and n-grams with stop-words.')
        logging.debug('stop_words=' + str(stop))
        
        # Filter out any token containing a stop-word
        self.corpus = [[token for token in document
                            if all(token_part not in stop 
                                   for token_part in token.split())
                       ] for document in self.corpus]
        return
    
    
    # Prepare the corpus for topic modelling
    #
    def preprocess(self):
        logging.info('Pipeline step 2: Preprocessing')
        self.tokenize()
        self.normalize()
        self.filter_length(min_strlen=3)
        self.filter_match(pattern=r'[A-Za-z]+')
        self.lowercase()
        self.add_n_grams(n=3)
        self.remove_stop_words(stop=stopwords.words('english') + ['use', 'also'])
        return
    
    
    # Transform lists of pre-processed tokens into an id: word frequency representation
    # params:
    #   no_below<integer>: The minimum amount of documents a word must appear in to be considered
    #   no_above<integer>: The maximum % of documents a word may appear in to be considered
    #
    def generate_dict(self, no_below=1, no_above=0.5):
        logging.info('Generating a dictionary of the corpus.')
        logging.debug('no_below=' + str(no_below))
        logging.debug('no_above=' + str(no_above))
        
        self.dictionary = corpora.Dictionary(self.corpus)

        # Filter out rare and common tokens
        self.dictionary.filter_extremes(no_below=no_below, no_above=no_above)
        return
        
    
    # Transforms lists of pre-processed tokens into a bag of words representation
    #
    def generate_bow(self):
        logging.info('Generating a Bag of Words representation of the corpus.')
        
        # Generate the bag of words
        self.corpus = [self.dictionary.doc2bow(documents) for documents in self.corpus]

        logging.debug('Number of unique tokens: ' + str(len(self.dictionary)))
        logging.debug('Number of documents: ' + str(len(self.corpus)))
        return

    
    # Train an LDA model on the corpus
    # params:
    #   num_topics<integer>: The amount of hidden topics in the corpus
    #   batch_size<integer>: The amount of documents to be processed at a time
    #   epochs<integer>: The amount of complete passes through the dataset before completing training
    #   iterations<integer>: Maximum iterations on the corpus before inferring a topic distribution
    #   eval_every<boolean>: Evaluate the log perplexity of the model (2x hit to training time)
    #
    def train_LDA(self, num_topics=5, batch_size=1000, epochs=10, iterations=400, eval_every=None):
        logging.info('Training the LDA model.')
        logging.debug('num_topics=' + str(num_topics))
        logging.debug('batch_size=' + str(batch_size))
        logging.debug('epochs=' + str(epochs))
        logging.debug('iterations=' + str(iterations))
        logging.debug('eval_every=' + str(eval_every))
        
        self.model = gensim.models.ldamodel.LdaModel(
            corpus=self.corpus, 
            id2word=self.dictionary, 
            num_topics=num_topics, 
            passes=epochs, 
            iterations=iterations, 
            chunksize=batch_size,
            alpha='auto', eta='auto', 
            eval_every=eval_every
        )
        for (i, z) in zip(range(num_topics), self.model.print_topics(num_words=num_topics)):
            logging.debug('Topic #' + str(i) + ": " + str(z))
        return
    
    
    # Save a frozen, trained topic model to disk
    # params:
    #   path<string>: The path to the save location for the model
    #
    def save_model(self, path='model.gensim'):
        logging.info('Saving the current model at: ' + str(path))
        self.model.save(path)
        return
    
    
    # Retrieve set of topics for a document from a trained LDA model
    # params:
    #   document<integer>: The index of the document for which topics will be retrieved
    # returns: <type>: TODO
    #
    def get_document_topics(self, document=0):
        return self.model.get_document_topics(self.corpus[document])
    
    
    # Calculate the topic coherence for the model
    # This is the sum of topic coherences of all topics, divided by the number of topics
    # params:
    #   num_topics<integer>: The amount of hidden topics in the corpus
    # returns: <float>: The average topic coherence for this model 
    #
    def get_coherence(self, num_topics=5):
        top_topics = self.model.top_topics(self.corpus)
        avg_topic_coherence = sum([z[1] for z in top_topics]) / num_topics
        logging.debug('Average topic coherence: ' + str(avg_topic_coherence))
        return avg_topic_coherence
    
    
    # Pretty print the top topics for this model
    #
    def print_topics(self):
        pprint(self.model.top_topics(self.corpus))
        return

    
    # Generate an HTML page to visualize the top topic distrubutions as 2D vectors
    # params:
    #   path<string>: The the path to the location where this HTML page should be saved
    #   save<boolean>: Save the HTML page or no
    #
    def generate_visual_LDA(self, path='lda-vis-data.html', save=False):
        logging.info('Generating an HTML page to display the LDA topic distributions...')
        lda_vis_data = pyLDAvis.gensim.prepare(self.model, self.corpus, self.dictionary)
        pyLDAvis.save_html(lda_vis_data, path)
        pyLDAvis.show(lda_vis_data)
        return

In [6]:
# (1) IMPORT DATASET
# Dataset is from https://www.kaggle.com/allen-institute-for-ai/CORD-19-research-challenge?select=metadata.csv
data = pd.read_csv('../data/metadata.csv', low_memory = False)
keep_columns = ['publish_time', 'journal', 'abstract']

# Select the relevant columns and rows from the dataset
new_data = data[keep_columns].dropna(subset=['abstract']).to_numpy()

# (2) PREPROCESS TEXT
all_abstracts = TopicModeler(new_data[:, -1])
all_abstracts.preprocess()

# (3) PROCESS TEXT
# (3.1) CREATE THE DICTIONARY AND BAG-OF-WORDS REPRESENTATION OF THE CORPUS
all_abstracts.generate_dict()
all_abstracts.generate_bow()

# (3.2) BUILD THE TOPIC MODEL
all_abstracts.train_LDA()
if not os.path.exists('../bin'):
    os.makedirs('../bin')
all_abstracts.save_model(path='../bin/model.gensim')
all_abstracts.get_document_topics()
all_abstracts.get_coherence()

# (4) ANALYZE THE DATA
all_abstracts.print_topics()
all_abstracts.generate_visual_LDA( path='../bin/lda-vis-data.html', save=True)

2020-06-10 10:59:51,811 : INFO : Initialized a TopicModeler with corpus size N=107032
2020-06-10 10:59:51,816 : INFO : Pipeline step 2: Preprocessing
2020-06-10 10:59:51,819 : INFO : Performing tokenization.
2020-06-10 11:12:35,722 : INFO : Performing normalization.
2020-06-10 11:12:35,726 : DEBUG : norm=lemma
2020-06-10 11:12:35,759 : INFO : Performing lemmatization.
2020-06-10 12:31:13,358 : INFO : Filtering out tokens that are out of the length bounds.
2020-06-10 12:31:13,553 : DEBUG : min_strlen=3
2020-06-10 12:31:13,570 : DEBUG : max_strlen=100
2020-06-10 12:31:54,479 : INFO : Filtering out tokens that match the regular expression: [A-Za-z]+
2020-06-10 12:33:46,966 : INFO : Converting all tokens to lowercase.
2020-06-10 12:34:08,228 : INFO : Performing normalization.
2020-06-10 12:34:08,432 : DEBUG : n=3
2020-06-10 12:34:08,445 : DEBUG : min_count=1
2020-06-10 12:34:08,449 : INFO : Adding 2-grams
2020-06-10 12:34:08,487 : INFO : collecting all words and their counts
2020-06-10 12:

2020-06-10 12:40:22,182 : DEBUG : rebuilding dictionary, shrinking gaps
2020-06-10 12:40:22,309 : INFO : resulting dictionary: Dictionary(100000 unique tokens: ['admission', 'affect', 'age group', 'bronchiolitis', 'clinical feature']...)
2020-06-10 12:40:22,352 : INFO : Generating a Bag of Words representation of the corpus.
2020-06-10 12:40:32,231 : DEBUG : Number of unique tokens: 100000
2020-06-10 12:40:32,232 : DEBUG : Number of documents: 107032
2020-06-10 12:40:32,233 : INFO : Training the LDA model.
2020-06-10 12:40:32,233 : DEBUG : num_topics=5
2020-06-10 12:40:32,234 : DEBUG : batch_size=1000
2020-06-10 12:40:32,235 : DEBUG : epochs=10
2020-06-10 12:40:32,235 : DEBUG : iterations=400
2020-06-10 12:40:32,237 : DEBUG : eval_every=None
2020-06-10 12:40:32,248 : INFO : using autotuned alpha, starting with [0.2, 0.2, 0.2, 0.2, 0.2]
2020-06-10 12:40:32,282 : INFO : using serial LDA version on this node
2020-06-10 12:40:32,378 : INFO : running online (multi-pass) LDA training, 5 topi

2020-06-10 12:40:41,201 : DEBUG : updating topics
2020-06-10 12:40:41,233 : INFO : merging changes from 1000 documents into a model of 107032 documents
2020-06-10 12:40:41,275 : INFO : topic #0 (0.220): 0.008*"cell" + 0.008*"infection" + 0.008*"disease" + 0.006*"virus" + 0.005*"study" + 0.005*"identify" + 0.005*"target" + 0.005*"protein" + 0.004*"potential" + 0.004*"may"
2020-06-10 12:40:41,277 : INFO : topic #1 (0.187): 0.010*"protein" + 0.010*"patient" + 0.010*"infection" + 0.009*"cell" + 0.007*"treatment" + 0.007*"mouse" + 0.006*"study" + 0.005*"increase" + 0.005*"human" + 0.005*"however"
2020-06-10 12:40:41,280 : INFO : topic #2 (0.171): 0.022*"patient" + 0.011*"study" + 0.008*"include" + 0.007*"infection" + 0.007*"case" + 0.006*"data" + 0.006*"background" + 0.005*"results" + 0.005*"day" + 0.004*"report"
2020-06-10 12:40:41,282 : INFO : topic #3 (0.038): 0.009*"patient" + 0.007*"identify" + 0.006*"gene" + 0.006*"test" + 0.006*"two" + 0.006*"group" + 0.005*"isolates" + 0.005*"virus"

2020-06-10 12:40:46,810 : INFO : optimized alpha [0.3319524, 0.26914424, 0.30177236, 0.05856656, 0.21521685]
2020-06-10 12:40:46,810 : DEBUG : updating topics
2020-06-10 12:40:46,842 : INFO : merging changes from 1000 documents into a model of 107032 documents
2020-06-10 12:40:46,878 : INFO : topic #0 (0.332): 0.009*"disease" + 0.006*"model" + 0.005*"study" + 0.005*"infection" + 0.005*"cell" + 0.005*"human" + 0.005*"system" + 0.004*"virus" + 0.004*"process" + 0.004*"protein"
2020-06-10 12:40:46,880 : INFO : topic #1 (0.269): 0.010*"cell" + 0.009*"infection" + 0.008*"patient" + 0.008*"treatment" + 0.007*"increase" + 0.006*"study" + 0.006*"protein" + 0.005*"disease" + 0.005*"may" + 0.005*"mouse"
2020-06-10 12:40:46,883 : INFO : topic #2 (0.302): 0.030*"patient" + 0.010*"study" + 0.008*"case" + 0.008*"infection" + 0.008*"include" + 0.006*"day" + 0.005*"results" + 0.005*"group" + 0.005*"treatment" + 0.005*"child"
2020-06-10 12:40:46,885 : INFO : topic #3 (0.059): 0.009*"group" + 0.008*"inf

2020-06-10 12:40:50,061 : DEBUG : 1000/1000 documents converged within 400 iterations
2020-06-10 12:40:50,068 : INFO : optimized alpha [0.5308988, 0.23736006, 0.42895392, 0.07178856, 0.24056698]
2020-06-10 12:40:50,068 : DEBUG : updating topics
2020-06-10 12:40:50,101 : INFO : merging changes from 1000 documents into a model of 107032 documents
2020-06-10 12:40:50,139 : INFO : topic #0 (0.531): 0.009*"model" + 0.007*"disease" + 0.007*"system" + 0.005*"approach" + 0.005*"many" + 0.005*"process" + 0.005*"new" + 0.004*"human" + 0.004*"study" + 0.004*"different"
2020-06-10 12:40:50,141 : INFO : topic #1 (0.237): 0.011*"cell" + 0.010*"infection" + 0.008*"treatment" + 0.008*"disease" + 0.007*"may" + 0.007*"patient" + 0.006*"increase" + 0.006*"protein" + 0.006*"study" + 0.005*"cause"
2020-06-10 12:40:50,144 : INFO : topic #2 (0.429): 0.019*"patient" + 0.009*"case" + 0.008*"study" + 0.008*"infection" + 0.007*"include" + 0.006*"data" + 0.005*"risk" + 0.005*"management" + 0.004*"treatment" + 0.0

...

2020-06-10 12:51:10,677 : INFO : topic #1 (0.183): 0.015*"cell" + 0.012*"infection" + 0.009*"treatment" + 0.008*"disease" + 0.007*"increase" + 0.006*"study" + 0.006*"may" + 0.006*"effect" + 0.005*"induce" + 0.005*"response"
2020-06-10 12:51:10,679 : INFO : topic #2 (0.245): 0.047*"patient" + 0.014*"results" + 0.014*"study" + 0.013*"group" + 0.010*"methods" + 0.010*"conclusions" + 0.010*"background" + 0.009*"compare" + 0.009*"treatment" + 0.008*"include"
2020-06-10 12:51:10,682 : INFO : topic #3 (0.187): 0.041*"covid-19" + 0.032*"patient" + 0.031*"infection" + 0.025*"case" + 0.018*"virus" + 0.014*"sars-cov-2" + 0.012*"child" + 0.012*"disease" + 0.010*"cause" + 0.009*"severe"
2020-06-10 12:51:10,685 : INFO : topic #4 (0.236): 0.023*"virus" + 0.013*"protein" + 0.009*"human" + 0.007*"viral" + 0.007*"infection" + 0.006*"study" + 0.006*"identify" + 0.006*"two" + 0.005*"detect" + 0.005*"gene"
2020-06-10 12:51:10,689 : INFO : topic diff=0.062057, rho=0.092437
2020-06-10 12:51:10,789 : INFO : P

2020-06-10 12:51:13,618 : INFO : topic #0 (0.469): 0.008*"disease" + 0.005*"model" + 0.005*"data" + 0.005*"outbreak" + 0.005*"include" + 0.005*"control" + 0.005*"new" + 0.005*"provide" + 0.004*"need" + 0.004*"covid-19"
2020-06-10 12:51:13,620 : INFO : topic #1 (0.184): 0.015*"cell" + 0.013*"infection" + 0.009*"treatment" + 0.008*"disease" + 0.007*"increase" + 0.006*"study" + 0.006*"may" + 0.006*"effect" + 0.006*"induce" + 0.005*"mouse"
2020-06-10 12:51:13,622 : INFO : topic #2 (0.241): 0.047*"patient" + 0.014*"study" + 0.014*"results" + 0.013*"group" + 0.010*"methods" + 0.010*"background" + 0.010*"conclusions" + 0.009*"compare" + 0.009*"treatment" + 0.007*"include"
2020-06-10 12:51:13,624 : INFO : topic #3 (0.187): 0.039*"covid-19" + 0.031*"patient" + 0.031*"infection" + 0.025*"case" + 0.018*"virus" + 0.013*"sars-cov-2" + 0.013*"child" + 0.012*"disease" + 0.011*"cause" + 0.009*"influenza"
2020-06-10 12:51:13,626 : INFO : topic #4 (0.250): 0.023*"virus" + 0.013*"protein" + 0.009*"human"

2020-06-10 12:51:16,310 : INFO : optimized alpha [0.47617233, 0.18566479, 0.23764767, 0.18599355, 0.26267508]
2020-06-10 12:51:16,311 : DEBUG : updating topics
2020-06-10 12:51:16,346 : INFO : merging changes from 1000 documents into a model of 107032 documents
2020-06-10 12:51:16,389 : INFO : topic #0 (0.476): 0.008*"disease" + 0.005*"model" + 0.005*"outbreak" + 0.005*"data" + 0.005*"include" + 0.005*"control" + 0.005*"need" + 0.004*"new" + 0.004*"provide" + 0.004*"covid-19"
2020-06-10 12:51:16,391 : INFO : topic #1 (0.186): 0.015*"cell" + 0.014*"infection" + 0.008*"disease" + 0.008*"treatment" + 0.007*"increase" + 0.006*"study" + 0.006*"may" + 0.006*"effect" + 0.005*"induce" + 0.005*"response"
2020-06-10 12:51:16,394 : INFO : topic #2 (0.238): 0.046*"patient" + 0.014*"study" + 0.014*"results" + 0.013*"group" + 0.010*"methods" + 0.010*"background" + 0.010*"conclusions" + 0.009*"compare" + 0.009*"treatment" + 0.007*"include"
2020-06-10 12:51:16,396 : INFO : topic #3 (0.186): 0.036*"cov

2020-06-10 12:51:18,791 : INFO : PROGRESS: pass 9, at document #105000/107032
2020-06-10 12:51:18,792 : DEBUG : performing inference on a chunk of 1000 documents
2020-06-10 12:51:19,142 : DEBUG : 1000/1000 documents converged within 400 iterations
2020-06-10 12:51:19,148 : INFO : optimized alpha [0.48541808, 0.18567847, 0.23772715, 0.18697116, 0.26912308]
2020-06-10 12:51:19,149 : DEBUG : updating topics
2020-06-10 12:51:19,179 : INFO : merging changes from 1000 documents into a model of 107032 documents
2020-06-10 12:51:19,215 : INFO : topic #0 (0.485): 0.008*"disease" + 0.005*"data" + 0.005*"include" + 0.005*"model" + 0.005*"outbreak" + 0.005*"covid-19" + 0.005*"need" + 0.005*"control" + 0.005*"provide" + 0.004*"new"
2020-06-10 12:51:19,217 : INFO : topic #1 (0.186): 0.015*"cell" + 0.015*"infection" + 0.008*"disease" + 0.008*"treatment" + 0.006*"increase" + 0.006*"study" + 0.006*"may" + 0.006*"effect" + 0.005*"induce" + 0.005*"response"
2020-06-10 12:51:19,219 : INFO : topic #2 (0.23

FileExistsError: [WinError 183] Cannot create a file when that file already exists: '../bin'

In [ ]:
if not os.path.exists('../bin'):
    os.makedirs('../bin')
all_abstracts.save_model(path='../bin/model.gensim')
all_abstracts.get_document_topics()
all_abstracts.get_coherence()

# (4) ANALYZE THE DATA
all_abstracts.print_topics()
all_abstracts.generate_visual_LDA( path='../bin/lda-vis-data.html', save=True)

2020-06-10 12:53:46,055 : INFO : Saving the current model at: ../bin/model.gensim
2020-06-10 12:53:46,058 : INFO : saving LdaState object under ../bin/model.gensim.state, separately None
2020-06-10 12:53:46,067 : DEBUG : {'uri': '../bin/model.gensim.state', 'mode': 'wb', 'buffering': -1, 'encoding': None, 'errors': None, 'newline': None, 'closefd': True, 'opener': None, 'ignore_ext': False, 'transport_params': None}
2020-06-10 12:53:46,101 : INFO : saved ../bin/model.gensim.state
2020-06-10 12:53:46,102 : DEBUG : {'uri': '../bin/model.gensim.id2word', 'mode': 'wb', 'buffering': -1, 'encoding': None, 'errors': None, 'newline': None, 'closefd': True, 'opener': None, 'ignore_ext': False, 'transport_params': None}
2020-06-10 12:53:48,422 : INFO : saving LdaModel object under ../bin/model.gensim, separately ['expElogbeta', 'sstats']
2020-06-10 12:53:48,424 : INFO : storing np array 'expElogbeta' to ../bin/model.gensim.expElogbeta.npy
2020-06-10 12:53:48,432 : INFO : not storing attribute st

2020-06-10 12:53:50,264 : INFO : CorpusAccumulator accumulated stats from 75000 documents
2020-06-10 12:53:50,284 : INFO : CorpusAccumulator accumulated stats from 76000 documents
2020-06-10 12:53:50,305 : INFO : CorpusAccumulator accumulated stats from 77000 documents
2020-06-10 12:53:50,328 : INFO : CorpusAccumulator accumulated stats from 78000 documents
2020-06-10 12:53:50,359 : INFO : CorpusAccumulator accumulated stats from 79000 documents
2020-06-10 12:53:50,385 : INFO : CorpusAccumulator accumulated stats from 80000 documents
2020-06-10 12:53:50,409 : INFO : CorpusAccumulator accumulated stats from 81000 documents
2020-06-10 12:53:50,435 : INFO : CorpusAccumulator accumulated stats from 82000 documents
2020-06-10 12:53:50,459 : INFO : CorpusAccumulator accumulated stats from 83000 documents
2020-06-10 12:53:50,481 : INFO : CorpusAccumulator accumulated stats from 84000 documents
2020-06-10 12:53:50,501 : INFO : CorpusAccumulator accumulated stats from 85000 documents
2020-06-10

2020-06-10 12:53:52,759 : INFO : CorpusAccumulator accumulated stats from 57000 documents
2020-06-10 12:53:52,778 : INFO : CorpusAccumulator accumulated stats from 58000 documents
2020-06-10 12:53:52,798 : INFO : CorpusAccumulator accumulated stats from 59000 documents
2020-06-10 12:53:52,816 : INFO : CorpusAccumulator accumulated stats from 60000 documents
2020-06-10 12:53:52,833 : INFO : CorpusAccumulator accumulated stats from 61000 documents
2020-06-10 12:53:52,850 : INFO : CorpusAccumulator accumulated stats from 62000 documents
2020-06-10 12:53:52,865 : INFO : CorpusAccumulator accumulated stats from 63000 documents
2020-06-10 12:53:52,881 : INFO : CorpusAccumulator accumulated stats from 64000 documents
2020-06-10 12:53:52,898 : INFO : CorpusAccumulator accumulated stats from 65000 documents
2020-06-10 12:53:52,915 : INFO : CorpusAccumulator accumulated stats from 66000 documents
2020-06-10 12:53:52,935 : INFO : CorpusAccumulator accumulated stats from 67000 documents
2020-06-10

[([(0.04510732, 'patient'),
   (0.01797238, 'study'),
   (0.013804931, 'results'),
   (0.012888719, 'group'),
   (0.0102352025, 'conclusions'),
   (0.0099891415, 'background'),
   (0.009665523, 'methods'),
   (0.008445704, 'compare'),
   (0.008205449, 'treatment'),
   (0.007703599, 'include'),
   (0.005990562, 'day'),
   (0.0058516236, 'conclusion'),
   (0.005424666, 'data'),
   (0.005404673, 'risk'),
   (0.0053849183, 'perform'),
   (0.0052326242, 'evaluate'),
   (0.005022992, 'high'),
   (0.004787081, 'clinical'),
   (0.004485149, 'report'),
   (0.0043432396, 'increase')],
  -1.6635672785641709),
 ([(0.0077866716, 'disease'),
   (0.0053261006, 'outbreak'),
   (0.0051394217, 'include'),
   (0.005131515, 'new'),
   (0.0049440027, 'model'),
   (0.004920904, 'data'),
   (0.004840013, 'covid-19'),
   (0.00471343, 'need'),
   (0.0046443483, 'provide'),
   (0.0044690454, 'research'),
   (0.0044274, 'pandemic'),
   (0.0043153255, 'control'),
   (0.0042083226, 'spread'),
   (0.0039413203, 'ep

2020-06-10 12:53:56,732 : DEBUG : performing inference on a chunk of 107032 documents
2020-06-10 12:54:35,075 : DEBUG : 107032/107032 documents converged within 400 iterations



Note: if you're in the IPython notebook, pyLDAvis.show() is not the best command
      to use. Consider using pyLDAvis.display(), or pyLDAvis.enable_notebook().
      See more information at http://pyLDAvis.github.io/quickstart.html .

You must interrupt the kernel to end this command

Serving to http://127.0.0.1:8889/    [Ctrl-C to exit]


127.0.0.1 - - [10/Jun/2020 12:56:16] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [10/Jun/2020 12:56:16] "GET /LDAvis.css HTTP/1.1" 200 -
127.0.0.1 - - [10/Jun/2020 12:56:16] "GET /d3.js HTTP/1.1" 200 -
127.0.0.1 - - [10/Jun/2020 12:56:17] code 404, message Not Found
127.0.0.1 - - [10/Jun/2020 12:56:17] "GET /favicon.ico HTTP/1.1" 404 -
----------------------------------------
Exception happened during processing of request from ('127.0.0.1', 63638)
Traceback (most recent call last):
  File "c:\users\whyve\appdata\local\programs\python\python37\lib\socketserver.py", line 316, in _handle_request_noblock
    self.process_request(request, client_address)
  File "c:\users\whyve\appdata\local\programs\python\python37\lib\socketserver.py", line 347, in process_request
    self.finish_request(request, client_address)
  File "c:\users\whyve\appdata\local\programs\python\python37\lib\socketserver.py", line 360, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "c:\use